<a href="https://colab.research.google.com/github/Lahiru-mta/Slum-Classification/blob/main/NDBI_NDVI_Sentinel_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Get authorize access by Earth Engine
import ee
# Trigger the authentication flow.
ee.Authenticate()

# Initialize the library.
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=keFwDpHV_nmxScVyHCoVyotQq95Jpmf8jstesszynI4&tc=yC8SBkiCzCsxW5sToMxNROIlFdW8fAp7S76Q-bZDGU4&cc=ggmXuqsaGnvrZ3am8hdLSKQHZ8hk1LpvrINH6jBOO-I

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1ARtbsJrOQjPKPSZhVfGhj8YfzdQQAtHCNiYSyxYwGatPpAhbk_EdBRfeKwo

Successfully saved authorization token.


In [ ]:
#@title Connect to Google Drive
#@markdown (To save outputs in the drive)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#@title Import dependencies
!pip install geemap --quiet
!pip install pydub --quiet

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.stats import norm, gamma, f, chi2
import IPython.display as disp
from IPython.display import clear_output 
from IPython.utils import io
import ipywidgets as widgets
%matplotlib inline
from datetime import datetime, timedelta
import gdown
import geemap
import time

from tqdm.notebook import tqdm

import shutil
import os
import zipfile
from pydub import AudioSegment
from pydub.utils import make_chunks

#for raster manupulation
!pip install rasterio --quiet
import rasterio
from rasterio.merge import merge
import glob
from rasterio.plot import show

#for filtering
!git clone https://github.com/adugnag/gee_s1_ard.git  --quiet
!pip install geemap --quiet

from importlib.machinery import SourceFileLoader

bnc = SourceFileLoader("border_noise_correction", "/content/gee_s1_ard/python-api/border_noise_correction.py").load_module()
sf = SourceFileLoader("speckle_filter", "/content/gee_s1_ard/python-api/speckle_filter.py").load_module()
trf = SourceFileLoader("terrain_flattening", "/content/gee_s1_ard/python-api/terrain_flattening.py").load_module()
wp = SourceFileLoader("wrapper", "/content/gee_s1_ard/python-api/wrapper.py").load_module()
helper = SourceFileLoader("helper", "/content/gee_s1_ard/python-api/helper.py").load_module()

clear_output() 

In [ ]:
#@title Functions

def date_from_img_name (img_1_name):
  name = img_1_name
  date_str = name[4:8] + '-' + name[8:10] + '-' + name[10:]
  date = datetime.fromisoformat(date_str).date()
  return date

def tometers(degrees):
  return(degrees * 111139)

def todegrees(meters):
  return(meters / 111139)

def fn_no_points(p_p_sqr_km, aoi):
  return (int(aoi.area(5).getInfo() * p_p_sqr_km / 1000000))

def find(seq,item):
    start_at = -1
    locs = []
    while True:
        try:
            loc = seq.index(item,start_at+1)
        except ValueError:
            break
        else:
            locs.append(loc)
            start_at = loc
    return locs

def rectangle(lon_1, lon_2, lat_1, lat_2):
    AOI_coordinates =  [[lon_1, lat_2],
                        [lon_1, lat_1],
                        [lon_2, lat_1],
                        [lon_2, lat_2],
                        [lon_1, lat_2]]

    aoi = ee.Geometry.Polygon(AOI_coordinates)
    return aoi

def apply_cld_shdw_mask(img):
    # Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
    not_cld_shdw = img.select('cloudmask').Not()

    # Subset reflectance bands and update their masks, return the result.
    return img.select('B.*').updateMask(not_cld_shdw)

def get_s2_sr_cld_col(aoi, start_date, end_date):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
        .filterBounds(aoi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
        .filterBounds(aoi)
        .filterDate(start_date, end_date))

    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))

def add_cloud_bands(img):
    # Get s2cloudless image, subset the probability band.
    cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

    # Condition s2cloudless by the probability threshold value.
    is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

    # Add the cloud probability layer and cloud mask as image bands.
    return img.addBands(ee.Image([cld_prb, is_cloud]))

def add_shadow_bands(img):
    # Identify water pixels from the SCL band.
    not_water = img.select('SCL').neq(6)

    # Identify dark NIR pixels that are not water (potential cloud shadow pixels).
    SR_BAND_SCALE = 1e4
    dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

    # Determine the direction to project cloud shadow from clouds (assumes UTM projection).
    shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

    # Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
    cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
        .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
        .select('distance')
        .mask()
        .rename('cloud_transform'))

    # Identify the intersection of dark pixels with cloud shadow projection.
    shadows = cld_proj.multiply(dark_pixels).rename('shadows')

    # Add dark pixels, cloud projection, and identified shadows as image bands.
    return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))

def add_cld_shdw_mask(img):
    # Add cloud component bands.
    img_cloud = add_cloud_bands(img)

    # Add cloud shadow component bands.
    img_cloud_shadow = add_shadow_bands(img_cloud)

    # Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
    is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

    # Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
    # 20 m scale is for speed, and assumes clouds don't require 10 m precision.
    is_cld_shdw = (is_cld_shdw.focalMin(2).focalMax(BUFFER*2/20)
        .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
        .rename('cloudmask'))

    # Add the final cloud-shadow mask to the image.
    return img_cloud_shadow.addBands(is_cld_shdw)
    # return img.addBands(is_cld_shdw)

def addNDBI(image):
  ndbi = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
  return image.addBands(ndbi)

def addNDVI(image):
  ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
  return image.addBands(ndvi)


In [ ]:
#@title Import the shape file of the Study area
#@markdown paste the Google Drive Shared link of the Zip below ex: " https://drive.google.com/file/d/1QKimP8DQl7cgHJ8SpYitwTOwvPGdu-lt/view?usp=sharing "
ZIP_link = 'https://drive.google.com/file/d/1SqCPF7_jpz0UmsUEtFp579ZMvDezvf4W/view?usp=sharing'#@param{type : "string"}
# 'https://drive.google.com/uc?id=1P3EFH5v8pOkJOUbDEmB_EpmqIbrHUZHk'
url = 'https://drive.google.com/uc?id=' + ZIP_link[32:65]
File_name = 'field_data' #param{type : "string"}
#@markdown *  ***please make sure the shape file is in WGS84 projection***
out = File_name +'.zip'

d_file_name = gdown.download(url, out, quiet=True)

zip_file = '/content/'+out
path = '/content/'+ File_name
zip_ref = zipfile.ZipFile(zip_file)
try:
    os.mkdir(path)
except:
    print('')
zip_ref.extractall(path)
zip_ref.close()

arr = os.listdir(path)
arr2=[]
for i in range(len(arr)):
  if arr[i].endswith('shp'):
    arr2.append(arr[i])
# print(arr2)

if len(arr2)>1:

  print(d_file_name+' contains '+str(len(arr2))+' shape files\n'
          + 'Select the required shape file below')

  shape_file = widgets.Dropdown(    options=arr2,    value=arr2[0],    description='Shape File:')
  display(shape_file)
else:
  shape_file = arr2[0]

# https://github.com/giswqs/geemap/blob/master/examples/notebooks/10_shapefiles.ipynb
# Refer the following code as well
# https://colab.research.google.com/github/csaybar/EEwPython/blob/master/4_features.ipynb


In [ ]:
#@title Parameters
#@markdown Parameters for the sentinel 1 image collection
Start_Date = '2019-01-01' #@param {type:"date"}
End_Date = '2020-01-01'  #@param {type:"date"}
Scale = 10  #param {type : 'slider', min:0, max:90, step:10  }
padding = 2700 #@param {type : 'slider', min:0, max:10000, step:100  }

#markdown Valleys are taken less than above value
Max_tile_size_km = 10 #param {type : 'slider', min:0, max:50, step:5  }
Show_AOI_on_map = False #@param {type:"boolean"}

start_date = Start_Date
end_date = End_Date
scale = Scale
padding_d = todegrees(padding)
tile_size = todegrees(Max_tile_size_km * 1000)
Show_on_Map = Show_AOI_on_map


print('\r','Collecting Data', end = '')

#@title Create an AOI

if len(arr2)>1:
  paddy_shp = path +'/'+ shape_file.value
else:
  paddy_shp = path +'/'+ shape_file
paddy = geemap.shp_to_ee(paddy_shp)

# paddy = ee.FeatureCollection("USDOS/LSIB_SIMPLE/2017").filterMetadata("country_co", "equals", "CE")

print('\r','Collecting Data.', end = '')

bound_coordinates = paddy.geometry().bounds().coordinates().getInfo()[0]
Upper_left = [a - padding_d for a in bound_coordinates[0]]
Bottom_right = [b + padding_d for b in bound_coordinates[2]]

lon_1 = Upper_left[0]
lon_2 = Bottom_right[0]
lat_1 = Upper_left[1]
lat_2 = Bottom_right[1]

AOI_coordinates =  [[lon_1, lat_2],
                    [lon_1, lat_1],
                    [lon_2, lat_1],
                    [lon_2, lat_2],
                    [lon_1, lat_2]]
      
print('\r','Collecting Data..', end = '')

aoi = ee.Geometry.Polygon(AOI_coordinates)

center = [(lat_1+lat_2)/2,(lon_1+lon_2)/2]
if Show_on_Map == True:
  Map = geemap.Map(center = (center[0], center[1]), zoom=12)
  Map.addLayer(aoi,{},'aoi')
  Map.addLayer(paddy, {}, 'paddy')
  display(Map)

print('\r','Collecting Data...', end = '')

AOI = aoi
START_DATE = start_date
END_DATE = end_date
CLOUD_FILTER = 80
CLD_PRB_THRESH = 50
NIR_DRK_THRESH = 0.15
CLD_PRJ_DIST = 1
BUFFER = 50

In [ ]:
#@title Process

s2_sr_cld_col = get_s2_sr_cld_col(aoi, START_DATE, END_DATE)
s2_sr_median = (s2_sr_cld_col.map(add_cld_shdw_mask)
                             .map(apply_cld_shdw_mask)
                             .median())
s2_sr_ndbi = addNDBI(s2_sr_median).clip(aoi)
s2_sr_ndbi_ndvi = addNDVI(s2_sr_median).clip(aoi)

In [ ]:
#@title Display result

Map = geemap.Map(center = (center[0], center[1]), zoom=12)
Map.addLayer(s2_sr_ndbi, {'bands': ['NDBI'], 'min':-1,'max':1},'NDBI')
# Map.addLayer(s2_sr_median,
#                 {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 2500, 'gamma': 1.1},
#                 'S2 cloud-free mosaic')
display(Map)

In [ ]:
#@title Export


n_cols = int((lon_2 - lon_1) / tile_size) + 1
n_rows = int((lat_2 - lat_1) / tile_size) + 1
n_tiles = n_cols * n_rows
n_tiles
e_0 = lon_1
e = [e_0]
for aa in range (n_cols):
  globals()['e_%s' % str(aa+1)] = e_0 + (lon_2 - lon_1)/n_cols * (aa+1)
  e.append(globals()['e_%s' % str(aa+1)])
# e
n_0 = lat_1
n = [n_0]
for ab in range (n_rows):
  globals()['e_%s' % str(ab+1)] = n_0 + (lat_2 - lat_1)/n_rows * (ab+1)
  n.append(globals()['e_%s' % str(ab+1)])
# n
ac = 0
aois = []
for ae in range (len(e)-1):
  for an in range (len(n)-1):
      globals()['aoi_%s' % ac] = rectangle(e[ae],e[ae+1],n[an],n[an+1])
      aois.append('aoi_' + str(ac))
      ac = ac +1

print ('\n',len(aois), 'tiles created')

#@title Export Data 
Export_to = '/content/drive/MyDrive/geeethya_para_5' #@param{type : "string"}
folder = Export_to + '/tiles_%s' %start_date

created_path = os.path.isdir(folder)
if created_path == True:
    print('Folder already exists')

else:

  try:
      os.mkdir(Export_to)
  except:
      print('path not created')

  try:
      os.mkdir(folder)
  except:
      print('folder not created')

created_path = os.path.isdir(folder)
if created_path == False:
    print('Something went wrong please create the path manualy and come back')


for roi in aois:
  c_aoi = globals()[roi]
  path = folder+'/'+start_date+'_'+roi+'.tif'
  # with io.capture_output() as captured:
  # print(path)
  geemap.ee_export_image(result, filename=path, scale=scale, region=c_aoi, file_per_band=False)


print ('%s TIFF files were saved at %s/' %(start_date,folder))

path2 = Export_to+'/mosaic_%s.tif' %start_date
search_criteria = "*aoi*.tif"
q = os.path.join(folder, search_criteria)
dem_fps = glob.glob(q)
src_files_to_mosaic = []
for fp in dem_fps:
  src = rasterio.open(fp)
  src_files_to_mosaic.append(src)
mosaic, out_trans = merge(src_files_to_mosaic)

out_meta = src.meta.copy()

out_meta.update({"driver": "GTiff",
              "height": mosaic.shape[1],
              "width": mosaic.shape[2],
              "transform": out_trans,
              }
              )

with rasterio.open(path2, "w", **out_meta) as dest:
  dest.write(mosaic)

print ('\r','Result was saved at %s ' %path2 , end = '')


 6 tiles created
Folder already exists
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/geeethya_para_5/tiles_2019-01-01/2019-01-01_aoi_0.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/geeethya_para_5/tiles_2019-01-01/2019-01-01_aoi_1.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/geeethya_para_5/tiles_2019-01-01/2019-01-01_aoi_2.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/geeethya_para_5/tiles_2019-01-01/2019-01-01_aoi_3.tif
Generating URL ...
Please wait ...
Data downloaded to /content/drive/MyDrive/geeethya_para_5/tiles_2019-01-01/2019-01-01_aoi_4.tif
Generating URL ...
Please wait ...


Data downloaded to /content/drive/MyDrive/geeethya_para_5/tiles_2019-01-01/2019-01-01_aoi_5.tif
2019-01-01 TIFF files were saved at /content/drive/MyDrive/geeethya_para_5/tiles_2019-01-01/
 Result was saved at /content/drive/MyDrive/geeethya_para_5/mosaic_2019-01-01.tif 